<a href="https://colab.research.google.com/github/aqeel-12/Bantu-Language-Modeling/blob/NLP-Challenge-4/NLP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


reference link:
https://towardsdatascience.com/text-generation-using-n-gram-model-8d12d9802aa0

In [ ]:
import re
import os
import sys
import numpy as np
import time
import math


In [ ]:
def get_ngrams(text):
  # Finding ngrams
  chars_tokens = (n)*['<START>'] + text  
  ngrams_nextChars = [(tuple(chars_tokens[i:i+n]),chars_tokens[i+n]) for i in range(len(chars_tokens)-n)]
  return ngrams_nextChars    

def addVal_to_dictKey(D, key, val=1):
  if key in D:
    if type(val)==int:
      D[key] += val
    elif (type(val)==str) & (val!='<START>'):
      D[key].append(val)
  else:
    if type(val)==int:
      D[key] = val
    elif (type(val)==str) & (val!='<START>'):
      D[key] = [val]
  return D

def find_probs_of_dict(D):
  total = 0
  for key in D.keys():
    total += D[key]
  for key in D.keys():
    D[key] /= total
  return D 

def get_dict_from_list(L):
  D = {}
  for char in L:
    if char in D:
      D[char] += 1
    else:
      D[char] = 1
  return D

def max_of_dict(D):  
  maxProb = 0
  maxKey = ''
  for key, probVal in D.items():
    if probVal>maxProb:
      maxProb = probVal
      maxKey = key
  return (maxKey, maxProb)



In [ ]:
def load_data(filePath):
  data_cwe = open(filePath, 'r').read().lower()
  return data_cwe

def clean_data(dataset: str) -> list:
  """
  Function task : It will extract each and every character form the dataset which is an ASCII character without losing the order of the characters
  -> Input Parameters --------------------
  --> dataset : String of paragraph in text
  
  -> Output Parameters --------------------
  --> all_characters_in_data : List all the ASCII characters from the dataset without losing the order of characters
  --> data_clean : Paragraph in text after cleaning.
  """
  ASCII_Chars = ' !"\'(),-.0123456789:;?abcdefghijklmnopqrstuvwxyz'
  all_characters_in_data = re.findall(r"[%s]"%ASCII_Chars, dataset)
  # data_clean = "".join(all_characters_in_data)
  return all_characters_in_data


def char_probability(list_of_chars: list) -> dict:
  """
  Funciton task: 

  Input Parameters : 
  -> list_of_chars: 

  Output Parameter:
  -> char_prob : 
  """
  total_chat_count = len(list_of_chars)
  ASCII_Chars = ' !"\'(),-.0123456789:;?abcdefghijklmnopqrstuvwxyz'
  char_prob = {char:0 for char in ASCII_Chars}

  # Finding a number for each character how many times it is repeated in the given dataset
  for char in list_of_chars:
    char_prob[char] += 1
  
  # Finding probabilities of each character
  for char in char_prob.keys():
    char_prob[char] /= total_chat_count
  
  return char_prob

def get_cond_prob(next_chars_of_ngram_prob):
  for ngram in next_chars_of_ngram_prob:
    ngramProb = ngram_prob[ngram]
    poss_next_chars = next_chars_of_ngram_prob[ngram]
    for key in poss_next_chars:
      poss_next_chars[key] = poss_next_chars[key] * char_prob[key] * ngramProb
    next_chars_of_ngram_prob[ngram] = poss_next_chars
  return next_chars_of_ngram_prob

def fit(all_characters_in_data, n=15):

  char_prob = char_probability(all_characters_in_data)

  all_ngrams = []
  ngram_prob = {}
  next_chars_of_ngram_prob = {}

  # chars_tokens = (n)*['<START>'] + all_characters_in_data

  # ===================================================================
  # Finding ngrams, ngrams-count & ngrams-next_chars
  # Finding ngrams (15-grams) & next-chat
  ngrams_nextChars = get_ngrams(all_characters_in_data)
  
  for ngram, next_char in ngrams_nextChars:
    all_ngrams.append(ngram)
    ngram_prob = addVal_to_dictKey(ngram_prob, ngram, 1) # (D, key, val)
    next_chars_of_ngram_prob = addVal_to_dictKey(next_chars_of_ngram_prob, ngram, next_char) # (D, key, val)
  all_ngrams = list(set(all_ngrams))

  # ===================================================================
  # Finding the probabilities of each ngram-count
  ngram_prob = find_probs_of_dict(ngram_prob)
  
  # ===================================================================
  # Finding the probabilities of each possible next_chars of each ngrams
  for ngram in next_chars_of_ngram_prob:
    possible_chars = next_chars_of_ngram_prob[ngram]  # it is a list

    # Finding count of each character repeated in ngram

    possible_chars_dict_prob = get_dict_from_list(next_chars_of_ngram_prob[ngram])
    next_chars_of_ngram_prob[ngram] = find_probs_of_dict(possible_chars_dict_prob)
    
    next_chars_of_ngram_condProb = get_cond_prob(next_chars_of_ngram_prob)
  return char_prob, ngram_prob, next_chars_of_ngram_prob, next_chars_of_ngram_condProb # all_ngrams, 

def generate_next_char_of_ngram(ngram):
  ngram = tuple(ngram[-15:])
  poss_next_chars = next_chars_of_ngram_condProb[ngram]
  
  print("possible next characters --------------------- ")
  print(list(poss_next_chars.keys()))
  print("\n")

  result = max_of_dict(poss_next_chars)
  print(f"Next possible character : {result[0]} ({result[1]})")
  return result

def evaluate(ngram, char):
  ngram = tuple(ngram[-15:])
  poss_next_chars = next_chars_of_ngram_condProb[ngram]
  print("possible next characters --------------------- ")
  print(list(poss_next_chars.keys()))
  print("\n")
  result = max_of_dict(poss_next_chars)
  
  print(f"Next Character : {result[0]}")
  print("\n")

  # finding loss
  if char in poss_next_chars.keys():
    char_prob = poss_next_chars[char]
  else:
    char_prob = 0
  loss = abs(result[1]-char_prob)
  print("Loss : ", loss)
  return loss

def cross_entropy_loss(input_ngram: str):
  ngrams_nextChars = get_ngrams(list(input_ngram))
  total_loss = 0
  for ngram, char in ngrams_nextChars:
    if ngram in next_chars_of_ngram_condProb:
      poss_next_chars = next_chars_of_ngram_condProb[ngram]
      if char in poss_next_chars:
        total_loss += -math.log2(poss_next_chars[char])
      else:
        total_loss = 0
      total_loss = total_loss/len(input_ngram)
  return total_loss

**Training model with training dataset**

In [ ]:
from datetime import datetime
startTime = datetime.now()

filePath = "/content/drive/MyDrive/NLP/Assignment03/data/cwe-train.txt"
dataset = load_data(filePath)

all_characters_in_data = clean_data(dataset)
char_prob, ngram_prob, next_chars_of_ngram_prob, next_chars_of_ngram_condProb = fit(all_characters_in_data, 15)

endTime = datetime.now()
print("Code Running Time : ", endTime-startTime)

Code Running Time :  0:00:06.059846


**Generating character**

In [ ]:
input_ngram = tuple("munhu yoyose yo")
_ = generate_next_char_of_ngram(input_ngram)

possible next characters --------------------- 
['h', 'm', 'y', 'n', 'l', 'g', 'j', 't', 'k', 'i', 'd', 'p', 's', 'v', 'z', 'b']


Next possible character : n (2.7822353455746416e-06)


**Evaluate Model with actual next character**

In [ ]:
_ = evaluate(tuple("munhu yoyose yo"), 'h')

possible next characters --------------------- 
['h', 'm', 'y', 'n', 'l', 'g', 'j', 't', 'k', 'i', 'd', 'p', 's', 'v', 'z', 'b']


Next possible character : n


Loss :  2.4972467825798725e-06


**Cross Entropy**

In [ ]:
cross_entropy_loss("u yoyose yohauhokela ")

1.101662996406211

### **Test Dataset**

In [ ]:
filePath = "/content/drive/MyDrive/NLP/Assignment03/data/cwe-test.txt"
dataset_test = load_data(filePath)

test_all_characters_in_data = clean_data(dataset_test)
ngrams_nextChars = get_ngrams(test_all_characters_in_data)

In [ ]:
TotalLoss = 0
for ngram, char in ngrams_nextChars:
 TotalLoss =  cross_entropy_loss("".join(ngram))
print(TotalLoss) 

0.0


In [ ]:
cross_entropy_loss("munhu yoyose yohauhokela u")

0.9228443236821698